In [1]:
import requests
res = requests.get('http://foodmap.isti.cnr.it:9200', auth=('elastic', 'changeme'))
print(res.content)

{
  "name" : "0PsTttc",
  "cluster_name" : "foodmap-cluster",
  "cluster_uuid" : "2YIbFkvuRqaPWhEjuEYnwA",
  "version" : {
    "number" : "5.4.0",
    "build_hash" : "780f8c4",
    "build_date" : "2017-04-28T17:43:27.229Z",
    "build_snapshot" : false,
    "lucene_version" : "6.5.0"
  },
  "tagline" : "You Know, for Search"
}



### ES Auth

In [3]:
from elasticsearch import Elasticsearch
es = Elasticsearch(['foodmap.isti.cnr.it'], http_auth=('elastic', 'changeme'), port=9200)
print es

<Elasticsearch([{u'host': u'foodmap.isti.cnr.it'}])>


In [71]:
# res = es.search(index="agg", doc_type='count', size=100, body={"query": {"match_all": {}}})
# res

### Carto Auth

In [72]:
from carto.auth import APIKeyAuthClient
from carto.sql import SQLClient
from carto.exceptions import CartoException


USERNAME="hpclab"
APIKEY = "e97d6bbf73f61f1faa5a3bafb81c370bdc131de5"
USR_BASE_URL = "https://{user}.carto.com/".format(user=USERNAME)
auth_client = APIKeyAuthClient(api_key=APIKEY, base_url=USR_BASE_URL)

In [94]:
result = es.search(index='agg', doc_type='count', size=1000000, body={
    'query': {
        "match_all" : {}
    }
})


# lst = [x['_source']['country']for x in result['hits']['hits']]
lst = [x for x in result['hits']['hits']]
for l in lst[:10]:
    print l

{u'_score': 1.0, u'_type': u'count', u'_id': u'1115', u'_source': {u'category': u'breakfast burrito', u'date': 20170315, u'count': 35, u'country': u'united states'}, u'_index': u'agg'}
{u'_score': 1.0, u'_type': u'count', u'_id': u'1123', u'_source': {u'category': u'breakfast burrito', u'date': 20170313, u'count': 26, u'country': u'united states'}, u'_index': u'agg'}
{u'_score': 1.0, u'_type': u'count', u'_id': u'1124', u'_source': {u'category': u'breakfast burrito', u'date': 20170324, u'count': 25, u'country': u'united states'}, u'_index': u'agg'}
{u'_score': 1.0, u'_type': u'count', u'_id': u'1128', u'_source': {u'category': u'breakfast burrito', u'date': 20170320, u'count': 22, u'country': u'united states'}, u'_index': u'agg'}
{u'_score': 1.0, u'_type': u'count', u'_id': u'1131', u'_source': {u'category': u'breakfast burrito', u'date': 20170317, u'count': 20, u'country': u'united states'}, u'_index': u'agg'}
{u'_score': 1.0, u'_type': u'count', u'_id': u'1136', u'_source': {u'catego

### Resources

#### torque - frequency

In [ ]:
### from itertools import groupby

dateBegin = 0
dateEnd = 20180400
category = 'pizza'
section = 'gafnjlan3'

# query es
result = es.search(index='agg_day', doc_type='count', size=10000000, body={
    "query": {        
        "bool" : {                
            "filter": [
                {"term" : { "category.keyword" : str(category) }},
                {"range": {            
                    "date" : {            
                        "gte" : dateBegin,
                        "lte" : dateEnd
                    }            
                }}
            ]            
        }
    }
})

# parse the result
lst_country = [(x["_source"]['country'],x["_source"]['count'],x["_source"]['datetime_date']) for x in result['hits']['hits']]

sql = SQLClient(auth_client)
sql_str = "delete from frequency_torque where section = '{section}';".format(section=section)
sql.send(sql_str)

f0 = lambda x: x[0]
for key, group in groupby(sorted(lst_country, key=f0), f0):    
    for country_value in list(group):        
        print country_value    
        dic_value = {'country': unicode(key), 'value': country_value[1], 'date': country_value[2], 'section': section, 'category': category}        

        try:        
            insert_value = "'{country}', {value}, '{section}', '{category}', '{date}'".format(country=dic_value['country'], value=dic_value['value'],section=dic_value['section'], category=dic_value['category'], date=dic_value['date'])                
            sql_str = "insert into frequency_torque (country, value, section, category, date) values ({insert_value});".format(insert_value=insert_value)
            sql.send(sql_str)             
        except Exception as e:
            print "some error ocurred", e, sql_str


#### for testing

In [99]:
from itertools import groupby

dateBegin = 0
dateEnd = 20180400
category = 'pizza'
section = 'gafnjlan3'

result = es.search(index='agg', doc_type='count', size=10000000, body={
    "query": {        
        "bool" : {                
            "filter": [
                {"term" : { "category.keyword" : str(category) }},
                {"range": {            
                    "date" : {            
                        "gte" : dateBegin,
                        "lte" : dateEnd
                    }            
                }}
            ]            
        }
    }
})

lst_country = [(x["_source"]['country'],x["_source"]['count']) for x in result['hits']['hits']]
f0 = lambda x: x[0]
f1 = lambda x: x[1]

sql = SQLClient(auth_client)
sql_str = "delete from trend_value where section = '{section}';".format(section=section)
print sql_str
sql.send(sql_str)


for key, group in groupby(sorted(lst_country, key=f0), f0):
    dic_value = {'country': unicode(key), 'value': sum(map(f1, list(group))), 'section': section, 'category': category}
    
    print 'country', unicode(key), 'value', sum(map(f1, list(group)))
    
    try:        
        insert_value = "'{country}', {value}, '{section}', '{category}'".format(country=dic_value['country'], value=dic_value['value'],section=dic_value['section'], category=dic_value['category'])                
        sql_str = "insert into trend_value (country, value, section, category) values ({insert_value});".format(insert_value=insert_value)
        sql.send(sql_str)        
    except Exception as e:
        print "some error ocurred", e, sql_str

lst_country = list(set(lst_country))
print 'finished!'


delete from trend_value where section = 'gafnjlan3';
country afghanistan value 0
country albania value 0
country alemania value 0
country algeria value 0
country allemagne value 0
country angola value 0
country anguilla value 0
country antarctica value 0
country argentina value 0
country armenia value 0
country aruba value 0
country australia value 0
country austria value 0
country azerbaijan value 0
country bahamas value 0
country bahrain value 0
country bangladesh value 0
country barbados value 0
country belarus value 0
country belgique value 0
country belgium value 0
country belgië value 0
some error ocurred 'ascii' codec can't encode character u'\xeb' in position 5: ordinal not in range(128) insert into trend_value (country, value, section, category) values ('belgium', 142, 'gafnjlan3', 'pizza');
country bermuda value 0
country bhutan value 0
country bolivia value 0
country bonaire, saint eustatius and saba  value 0
country bosnia and herzegovina value 0
country brasil value 0
coun

In [68]:



    # print key,sum()

australia 7
austria 4
brazil 1
canada 6
colombia 2
cyprus 1
france 1
germany 9
greece 3
india 5
iraq 1
ireland 5
israel 2
italy 6
japan 1
kingdom of saudi arabia 1
lebanon 1
malaysia 1
nederland 1
netherlands 13
oman 2
romania 1
serbia 1
slovenia 2
south africa 1
sweden 2
turkey 34
türkiye 3
united arab emirates 1
united kingdom 21
united states 85
venezuela 2
поранешна југословенска република македонија 1
